## FER 2013 Sentiment Analysis using ResNet50

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation and normalization
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    "dataset/train",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    "dataset/test",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


## Load the ResNet50 model and customize it for your dataset.

In [2]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Load ResNet50 model.
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [3]:
# Add new classifications layers.
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

In [4]:
# Create model.
model = Model(inputs=base_model.input, outputs=predictions)

In [5]:
print(model.summary())

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [6]:
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.0001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

## Train the model.

In [7]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

'''Early stopping and model saving
patience : This parameter specifies how many epochs should be continued 
If there is no improvement in the metric during training.'''
callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True),
    ModelCheckpoint('resnet50.h5', monitor='val_loss', save_best_only=True)
]

# Start training.
steps_per_epoch = len(train_generator) // 15
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=15,
    batch_size=128,
    steps_per_epoch=steps_per_epoch,  
    callbacks=callbacks)

Epoch 1/15
59/59 [==============================] - 385s 7s/step - loss: 2.0520 - accuracy: 0.1811 - val_loss: 1.8164 - val_accuracy: 0.2471
Epoch 2/15
59/59 [==============================] - 376s 6s/step - loss: 1.8661 - accuracy: 0.2283 - val_loss: 1.8177 - val_accuracy: 0.2471
Epoch 3/15
59/59 [==============================] - 375s 6s/step - loss: 1.8554 - accuracy: 0.2177 - val_loss: 1.8178 - val_accuracy: 0.2473
Epoch 4/15
59/59 [==============================] - 378s 6s/step - loss: 1.8351 - accuracy: 0.2410 - val_loss: 1.8179 - val_accuracy: 0.2474
Epoch 5/15
59/59 [==============================] - 417s 7s/step - loss: 1.8477 - accuracy: 0.2082 - val_loss: 1.8193 - val_accuracy: 0.2477
Epoch 6/15
59/59 [==============================] - ETA: 0s - loss: 1.8288 - accuracy: 0.2272


KeyboardInterrupt



## I stopped the code because the val_accuracy increase for 6 epochs was less than 0.0008.